## Lecture 1 - Visualization of High Dimensional Data

In [214]:
import numpy as np
import pandas as pd
from nptyping import NDArray, Int, Shape
from sympy import symbols, init_printing, solve, Array, Symbol, linsolve, Sum
from sympy import tensorproduct as tp
import sympy as sp

### 4. Expectation and Covariance of a Random Vector

#### Covariance Matrix of a Random Vector

![Alt text](Images/covariance_random_vector.PNG)

### 5. Review: Empirical Mean and Covariance Matrix of a Vector Data Set I

#### Review

In [3]:
# Matrix
X: NDArray[Shape["4, 3"], Int] = np.array([
    [8, 4, 7],
    [2, 8, 1],
    [3, 1, 1],
    [9, 7, 4]
])

# Number of samples
n: int = X.shape[0]

# Number of features
d: int = X.shape[1]

#### A Formula for the Vector Mean

In [4]:
# Get the mean of each row
print("Manually Calculated Sample Mean: ", X.mean(axis=0))

# Column vector of ones
ones = np.ones((n, 1))

# Try to get the same result but using matrix multiplication. All elements
# next to the "ones" array will consist of the A matrix
print("Sample Mean with Matrix Multiplications: ", (X.T @ ones) / n)
print("The value of A in the problem is X.T / n")

Manually Calculated Sample Mean:  [5.5  5.   3.25]
Sample Mean with Matrix Multiplications:  [[5.5 ]
 [5.  ]
 [3.25]]
The value of A in the problem is X.T / n


#### The Sample Covariance for a Data Set of Vector

![Alt text](Images/sample_covariance.PNG)

In [5]:
# Re-calculate the sample mean
sample_mean = X.T @ ones / n

# Calculate the sum
sum = 0
for i in range(n):

    # Turn each Xi into a column vector
    # (At the beginning, it is said that each Xi is a column vector)
    Xi: NDArray[Shape["3, 1"], Int] = X[i, :].reshape(d, 1)

    # Process the sum
    sum += Xi @ Xi.T

print("Sum: ")
print(sum)

# Calculate the empirical covariance
empirical_covariance = ((1/n) * sum) - (sample_mean @ sample_mean.T)
print("Empirical Covariance: ")
print(empirical_covariance)

Sum: 
[[158 114  97]
 [114 130  65]
 [ 97  65  67]]
Empirical Covariance: 
[[9.25   1.     6.375 ]
 [1.     7.5    0.    ]
 [6.375  0.     6.1875]]


#### Formula for the Empirical Covariance Matrix

In [6]:
# Identity matrix of n x n and d x d
In = np.identity(n)
Id = np.identity(d)

# Ones column vector
ones = np.ones((n, 1))

# ================== OPTION 1 ================== #

option_1 = (1/n) * X.T @ (In - ones.T @ ones) @ X
print("Option 1: ", np.array_equal(option_1, empirical_covariance))

# ================== OPTION 2 ================== #

option_2 = (1/n) * X.T @ (In - (1/n) * ones @ ones.T) @ X
print("Option 2: ", np.array_equal(option_2, empirical_covariance))

# ================== OPTION 3 ================== #

# Fails due to dimensional reasons. 
# The matrix multiplication is not possible
try:
    option_3 = (1/n) * X @ (Id - (1/d) * ones @ ones.T) @ X.T
except:
    print("Option 3: ", False)

# ================== OPTION 4 ================== #

# Fails due to dimensional reasons.
# The matrix multiplication is not possible
try:
    option_4 = (1/n) * X @ (In - (1/n) * ones @ ones.T) @ X.T
except:
    print("Option 4: ", False)

Option 1:  False
Option 2:  True
Option 3:  False
Option 4:  False


### 6. Empirical Mean and Covariance Matrix of a Vector Data Set II

#### Matrix Products Involving Outer Products

In [7]:
# Vector where the sum of the elements is 0
x: NDArray[Shape["3,1"], Int] = np.array([
    [1],
    [2],
    [-3],
])

# Column unit vector
ones = np.ones((3, 1))

# What is this expression equal to?
print((ones @ ones.T) @ x)

[[0.]
 [0.]
 [0.]]


#### An Orthogonal Projection Matrix

In [8]:
x = np.array([2, -1, -2]).T

# Identity matrix and unit column vector
In = np.identity(3)
ones = np.ones((3, 1))

# Calculate H
H = In - (1/3) * ones @ ones.T

# Get Hx
print("Hx: ", H @ x)

# Get H^2x
print("H^2x: ", H @ H @ x)

Hx:  [ 2.33333333 -0.66666667 -1.66666667]
H^2x:  [ 2.33333333 -0.66666667 -1.66666667]


### 7. Measuring the Spread of a Point Cloud

#### Projection onto a Line

![Alt text](Images/vector_projection.PNG)

In [9]:
# Dimensions
d = 2

# The vector onto which we project
u = np.array([[1, 2]]).T * (1/np.sqrt(5))

# Dataset
x1 = np.array([[1, 2]]).T
x2 = np.array([[3, 4]]).T
x3 = np.array([[-1, 0]]).T

# See shapes
print("Shapes")
print("x1: ", x1.shape)
print("u: ", u.shape)
print()

# Find the projection of each sample onto u
print("Projection of x1 onto u: ", (u @ x1.T) @ u)
print("Projection of x2 onto u: ", (u @ x2.T) @ u)
print("Projection of x3 onto u: ", (u @ x3.T) @ u)

Shapes
x1:  (2, 1)
u:  (2, 1)

Projection of x1 onto u:  [[1.]
 [2.]]
Projection of x2 onto u:  [[2.2]
 [4.4]]
Projection of x3 onto u:  [[-0.2]
 [-0.4]]


#### Empirical Variance of a Data Set in a Given Direction

In [10]:
# Empirical variance: The signed distance from the origin to the 
# endpoint of the projection

# Get the norm of all the u * Xi 
signed_distances = np.array([
    (u.T @ x1).item(),
    (u.T @ x2).item(),
    (u.T @ x3).item()
])
print("Signed distances: ", signed_distances)

# Get the empirical variance
print("Empirical Variance: ", np.var(signed_distances))

# Build a matrix with all the samples
X = np.vstack((x1.T, x2.T, x3.T))

# Re-calculate n and d
n = X.shape[0]
d = X.shape[1]

# Identity matrix and column vector of ones
In = np.identity(n)
ones = np.ones((n, 1))

# Calculate the empirical covariance
S = (1/n) * X.T @ (In - (1/n) * ones @ ones.T) @ X

# Get u^T S u
print("u^T S u: ", (u.T @ S @ u))


Signed distances:  [ 2.23606798  4.91934955 -0.4472136 ]
Empirical Variance:  4.8
u^T S u:  [[4.8]]


### 8. The Decomposition Theorem for Symmetric Matrices

#### Concept Check: Othogonal Matrices

In [11]:
# Identity matrix of d x d
d = 3
Id = np.identity(d)

# Check that the matrix is orthogonal 
condition_1 = np.array_equal(Id @ Id.T, Id)
condition_2 = np.array_equal(Id.T @ Id, Id)
print("P P^T = Id: ", condition_1)
print("P^T P = Id: ", condition_2)

# Is P orthogonal?
if condition_1 and condition_2:
    print("P is orthogonal")
else:
    print("P is not orthogonal")

# What is v1 * v2?
v1 = Id[:, 0]
v2 = Id[:, 1]
print("v1 * v2: ", v1 @ v2)

P P^T = Id:  True
P^T P = Id:  True
P is orthogonal
v1 * v2:  0.0


#### Eigenvectors and Eigenvalues of a Decomposition of a Symmetric Matrix

In [12]:

from sympy import Matrix

Id = np.identity(2)
P = Id.copy()

# Declare the positive value for lambda
lamb = Symbol("lambda", positive=True)

# Get the diagonal matrix of positive lambdas
D = Matrix([
    [lamb,    0],
    [   0, lamb]
])

# Get the value of PDP^T
A = tp(tp(P, D), P.T)

# Declare the elements of the v1 vector
v1_1, v1_2 = symbols("v1_1, v1_2")

# Vi is the ith column of P. V1 is the first column
v1 = Matrix([
    [v1_1],
    [v1_2]
])

# Get the value of A * v1
print(A * v1)
print("This is equal to A * v1")

Matrix([[1.0*lambda*v1_1], [1.0*lambda*v1_2]])
This is equal to A * v1


#### Concept Check: The Decomposition Theorem for Symmetric Matrices

![Alt text](Images/decomposition_theorem.PNG)

### 9. Principal Component Analysis (PCA)

#### Centering Data

In [24]:
# Dimensions
n = 2

# Identity matrix of n x n and column vector of ones
In = np.identity(n)
ones = np.ones((n, 1))

# Calculate the matrix H
H = In - (1/n) * ones @ ones.T
print("H: ", H)

# Calculate its square
H2 = H @ H
print("H^2: ", H2)

# Is H^2 = H?
if np.array_equal(H2, H):
    print("H^2 = H")
    print()
else:
    print("H^2 != H")
    print()

H:  [[ 0.5 -0.5]
 [-0.5  0.5]]
H^2:  [[ 0.5 -0.5]
 [-0.5  0.5]]
H^2 = H



In [29]:
# Sample data (centered around 0 with a sample mean of 0 in both dimensions)
# X = [
#   [x1, y1],
#   [x2, y2]
# ]
X = np.array([
    [0, 1],
    [0, -1]
])

# We calculate H (the projection matrix)
n = X.shape[0]
ones = np.ones((n, 1))
In = np.identity(n)
H = In - (1/n) * ones @ ones.T

# We calculate the empirical covariance matrix
# Note: Here we can use either "n-1" or "n" as the denominator for the first element.
# By using a different denominator you are simply scaling the result by a constant, so
# the result will be the same ultimately the same.
S = (1/(n-1)) * X.T @ H @ X
print("S: ", S)
print()

# Find the eigenvalues and eigenvectors of S
# (In other words, find pairs of scalar lambda and vector v such that Sv = lambda v)
eigenvalues, eigenvectors = np.linalg.eig(S)
print("Eigenvalues: ", eigenvalues)
print("Eigenvectors: ", eigenvectors)

S:  [[0. 0.]
 [0. 2.]]

Eigenvalues:  [0. 2.]
Eigenvectors:  [[1. 0.]
 [0. 1.]]


### 10. Conceptual Examples in 2 Dimensions

#### Conceptual Example II: 2 Data points in 2D

In [100]:
# Data set (centered around 0 with a sample mean of 0 in both dimensions)
X = np.array([
    [ 1,  1/2],
    [-1, -1/2]
])

# Find the projection matrix (H)
n = X.shape[0]
ones = np.ones((n, 1))
In = np.identity(n)
H = In - (1/n) * ones @ ones.T

# Calculate the empirical covariance matrix (S)
S = (1/(n-1)) * X.T @ H @ X
print("S: ", S)

# Find the eigenvalues of S
eigenvalues, _ = np.linalg.eig(S)
print("Eigenvalues: ", eigenvalues)

# Symbols
v1_1, v1_2 = symbols("v1_1, v1_2")
v2_1, v2_2 = symbols("v2_1, v2_2")

# Symbolic eigenvectors
v1 = Matrix([
    [v1_1],
    [v1_2]
])
v2 = Matrix([
    [v2_1],
    [v2_2]
])

# Find the set of equations that define the first eigenvector
# (S - lambda_1 * Id) * v_1 = 0
solutions1 = linsolve((S - eigenvalues[0] * In) * v1, (v1_1, v1_2))
print("Solutions for v1", solutions1)

# Find the second eigenvector by solving the following equation:
# (S - lambda_2 * Id) * v_2 = 0
solutions2 = linsolve((S - eigenvalues[1] * In) * v2, (v2_1, v2_2))
print("Solutions for v2", solutions2)

# NOTE: The equations above dont have an exact solution, they have actually an infinite
# set of solutions. This is because the eigenvectors are not unique. The thing that we
# need to pay attention is the relationship between each of the components in a solution.
# For example, above we have solutions for 
#   - v1: {2v1_2, v1_2} which means that the eigenvector is a scalar multiple of [2, 1].
#   - v2: {-0,5v2_2, v2_2} which means that the eigenvector is a scalar multiple of [-0.5, 1]
#     or [1, -2] if we divide by -0.5

S:  [[2.  1. ]
 [1.  0.5]]
Eigenvalues:  [2.5 0. ]
Solutions for v1 {(2.0*v1_2, v1_2)}
Solutions for v2 {(-0.5*v2_2, v2_2)}


#### Project onto PC1

In [124]:
# Get the first principal component defined above
PC1 = np.array([[2, 1]]).T

# Normalize the first principal component
PC1 = PC1 / np.linalg.norm(PC1)

# Get the projection of x1 and x2 onto the first principal 
# component (y1 and y2)
y = PC1.T @ X.T
print("Projection of the data points onto PC1: ", y)
print("First element is y1 (projection of x1 onto PC1) and second element is y2")

Projection of the data points onto PC1:  [[ 1.11803399 -1.11803399]]
First element is y1 (projection of x1 onto PC1) and second element is y2


### 11. (Optional) Conceptual Examples in 2 Dimensions

#### Conceptual Example III: 3 Data points in 2D

In [167]:
# The 3 2D data points
X = np.array([
    [ 0,  2],
    [ 1, -1],
    [-1, -1]
])

# Find the projection matrix (H)
n = X.shape[0]
ones = np.ones((n, 1))
In = np.identity(n)
H = In - (1/n) * ones @ ones.T

# Calculate the empirical covariance matrix (S)
S = (1/(n-1)) * X.T @ H @ X
print("S: ", S)

S:  [[ 1.00000000e+00  0.00000000e+00]
 [-1.11022302e-16  3.00000000e+00]]


#### Spectral Decomposition of the Covariance Matrix

In [168]:
# Get the eigenvalues of S
eigenvalues, eigenvectors = np.linalg.eig(S)
print("Eigenvalues: ", eigenvalues)
print("Normalized eigenvectors: ", eigenvectors)
print()

# Symbols
v1_1, v1_2, v2_1, v2_2 = symbols("v1_1, v1_2, v2_1, v2_2")
v1 = Matrix([
    [v1_1],
    [v1_2]
])
v2 = Matrix([
    [v2_1],
    [v2_2]
])

print("Manual calculation")
print("==================")

# Identity matrix of d x d
d = X.shape[1]
Id = np.identity(d)

# Find the set of equations that define the first eigenvector
# (S - lambda_1 * Id) * v_1 = 0
solutions1 = linsolve((S - eigenvalues[0] * Id) * v1, (v1_1, v1_2))
print("Solutions for v1", solutions1)

# Find the second eigenvector by solving the following equation:
# (S - lambda_2 * Id) * v_2 = 0
solutions2 = linsolve((S - eigenvalues[1] * Id) * v2, (v2_1, v2_2))
print("Solutions for v2", solutions2)

# Print the eigenvectors
print("Eigenvector 1 (based on solutions1): [0, 1]")
print("Eigenvector 2 (based on solutions2): [1, 0]")


Eigenvalues:  [3. 1.]
Normalized eigenvectors:  [[0.00000000e+00 1.00000000e+00]
 [1.00000000e+00 5.55111512e-17]]

Manual calculation
Solutions for v1 {(0, v1_2)}
Solutions for v2 {(1.8014398509482e+16*v2_2, v2_2)}
Eigenvector 1 (based on solutions1): [0, 1]
Eigenvector 2 (based on solutions2): [1, 0]


#### Output of PCA

In [172]:
# First principal component (normalized)
PC1 = np.array([[0, 1]]).T
PC1 = PC1 / np.linalg.norm(PC1)

# Calculate y1, y2, and y3, the signed length of the projections of the
# three data points onto the first principal component. Remember that the
# signed length of a projection is just "u * x" instead of "(u * x) * u" for
# a projection
y = PC1.T @ X.T
print("Projection of the data points onto PC1: ", y)

# Sample variance of y
var_y = np.var(y)
print("Sample variance of y: ", var_y)

Projection of the data points onto PC1:  [[ 2. -1. -1.]]
Sample variance of y:  2.0


The following section is done to prove that the transition matrix from X to the space formed by PC1 and PC2 consists of each of the normalized eigenvectors of S as a row of the matrix. This is part of the proof in the "output of PCA" section of part 12.

In [175]:
PC2 = np.array([[1, 0]]).T
PC2 = PC2 / np.linalg.norm(PC2)

# Transition matrix to have data points in PC1 and PC2
T = np.concatenate((PC1, PC2), axis=1)
print("Transition matrix", T)

# Project the data points onto PC1 and PC2
Y = T.T @ X.T
print("Projection onto the (PC1, PC2) space: ", Y)

Transition matrix [[0. 1.]
 [1. 0.]]
Projection onto the (PC1, PC2) space:  [[ 2. -1. -1.]
 [ 0.  1. -1.]]


### 12. Conceptual Examples Continued

#### Conceptual Example IV: 4 Data points in 2D

In [180]:
# 4 2D data points
X = np.array([
    [ 0,  2],
    [ 0, -2],
    [ 1,  1],
    [-1, -1],
])

# Get the projection matrix
n = X.shape[0]
ones = np.ones((n, 1))
In = np.identity(n)
H = In - (1/n) * ones @ ones.T 

# Calculate the empirical covariance matrix
# Note: you can also use (1/n) here and the results will be simpler and equally valid
S = (1/(n)) * X.T @ H @ X
print("S: ", S)

S:  [[0.5 0.5]
 [0.5 2.5]]


#### Spectral Decomposition of the Covariance Matrix

In [181]:
# Get the eigenvalues of S
eigenvalues, eigenvectors = np.linalg.eig(S)
print("Eigenvalues: ", eigenvalues)
print("Normalized eigenvectors: ", eigenvectors)
print()

# Symbols
v1_1, v1_2, v2_1, v2_2 = symbols("v1_1, v1_2, v2_1, v2_2")
v1 = Matrix([
    [v1_1],
    [v1_2]
])
v2 = Matrix([
    [v2_1],
    [v2_2]
])

print("Manual calculation")
print("==================")

# Identity matrix of d x d
d = X.shape[1]
Id = np.identity(d)

# Find the set of equations that define the first eigenvector
# (S - lambda_1 * Id) * v_1 = 0
solutions1 = linsolve((S - eigenvalues[0] * Id) * v1, (v1_1, v1_2))
print("Solutions for v1:", solutions1)

# Find the second eigenvector by solving the following equation:
# (S - lambda_2 * Id) * v_2 = 0
solutions2 = linsolve((S - eigenvalues[1] * Id) * v2, (v2_1, v2_2))
print("Solutions for v2", solutions2)

# Note: Sympy was unable to find the solutions for the eigenvectors,
# so the solution was to use the eigenvectors from wolfram alpha:
# v1 = [-2 + sqrt(5), 1] and v2 = [-2 - sqrt(5), 1]
# 
# Link: https://www.wolframalpha.com/input?i2d=true&i=%7B%7BDivide%5B2%2C3%5D%2CDivide%5B2%2C3%5D%7D%2C%7BDivide%5B2%2C3%5D%2CDivide%5B10%2C3%5D%7D%7D

Eigenvalues:  [0.38196601 2.61803399]
Normalized eigenvectors:  [[-0.97324899 -0.22975292]
 [ 0.22975292 -0.97324899]]

Manual calculation
Solutions for v1: {(0, 0)}
Solutions for v2 {(0, 0)}


### 13. Covariance versus Correlation

![Alt text](Images/correlation_random_vars.PNG)

![Alt text](Images/covariance-based-pca-vs-correlation-based-pca.PNG)

### 14. Total Variance

![Alt text](Images/total_variance.PNG)

Response from an assistant professor:
A correlation matrix has ones down the diagonal, so nothing interesting is happening there. Instead, PCA can be respond to off-diagonal elements, looking for some direction—not an axis—where strong correlations lead to higher variance. (Consider a two-dimensional case with perfectly correlated variables. PC1 would be the  diagonal.)

But what if all the variables are independent? The correlation matrix is . All eigenvalues are one, every vector is an eigenvector. No direction is better than any other!

In contrast, PCA can still do something useful with the covariance matrix in this case. PC1 will be along an axis, in the direction of whichever variable has the greatest variance. (I think this is a counterexample to a point from another thread, where if we normalize the data, we can't recover what PCA could have told us about the unnormalized data. It also illustrates a problem with generating all our test cases randomly: What are the chances we get a diagonal matrix? But that's an important corner case!)

### 15. Multidimensional Scaling (MDS)

![Alt text](Images/dimensionality_reduction_methods.PNG)

- distance matrix: Distance measure between points. Euclidean distance, manhattan distance, etc.
- dissimilarity matrix: Similarity measure between points. 1 - correlation, 1 - cosine similarity, etc.
- In the end MDS is just like PCA, but instead of using the covariance matrix to find the eigenvectors, we use the matrix $X X^T$. Due to the way how classical PCA operates, it technically operates on $X^T X$ (used for the covariance matrix).

![Alt text](Images/gram_matrix.PNG)

### 16. Solving MDS

#### Implementation of Simple Example

In [196]:
# 3 2D data points
X = np.array([
    [1, 1],
    [1, -1],
    [-1, 1]
])

# Get the gram matrix
B = X @ X.T
print("Gram matrix: ", B)

# Find the eigenvalues and eigenvectors of the gram matrix
# Note: Each column in the eigenvectors matrix is an eigenvector
eigenvalues, eigenvectors = np.linalg.eig(B)
print("Eigenvalues: ", eigenvalues)
print("Normalized eigenvectors: ", eigenvectors)
print()

# We construct the diagonal matrix "sigma 1" by choosing the
# q largest (q = 1 in this case, since that is the target dimensionality)
# eigenvalues and putting them in a diagonal matrix
sigma_1 = np.diag(eigenvalues[:1])
print("Sigma 1: ", sigma_1)

# We construct the matrix V_1 by choosing the q largest eigenvectors
# (V1 = [v1, v2, ..., vq], where v1 is a column vector)
V_1 = np.reshape(eigenvectors[:, 0], (3, 1))
print("V 1: ", V_1)

# Get the solution to the MDS problem (Y = V_1 * (sigma_1)^1/2)
Y = V_1 @ np.sqrt(sigma_1)
print("Solution to the MDS problem: ", Y)

Gram matrix:  [[ 2  0  0]
 [ 0  2 -2]
 [ 0 -2  2]]
Eigenvalues:  [4.0000000e+00 4.4408921e-16 2.0000000e+00]
Normalized eigenvectors:  [[ 0.          0.          1.        ]
 [-0.70710678  0.70710678  0.        ]
 [ 0.70710678  0.70710678  0.        ]]

Sigma 1:  [[4.]]
V 1:  [[ 0.        ]
 [-0.70710678]
 [ 0.70710678]]
Solution to the MDS problem:  [[ 0.        ]
 [-1.41421356]
 [ 1.41421356]]


### 17. Stochastic Neighbor Embedding (SNE) and t-distributed Stochastic Neighbor Embedding (t-SNE)

![Alt text](Images/sne_p_ij.PNG)
![Alt text](Images/sne_q_ij.PNG)

#### First Example: Distribution in the High Dimension

In [207]:
# All points are at a distance "A" of each other
A = Symbol("A", positive=True)

# Create the distance matrix
D = Matrix([
    [0, A, A],
    [A, 0, A],
    [A, A, 0]
])

# We need to calculate all p_ij: p_12, p_13, p_23
# All other p_ij are just the transpose of the other p_ij, so we just use the lower half
D_lower = D.lower_triangular()

from sympy import exp

# We get the sum of all non-zero elements in the lower triangular matrix
# (i.e. the sum of all p_ij)
total_sum = 0
for i in range(D_lower.shape[0]):
    for j in range(D_lower.shape[1]):
        if i > j:
            total_sum += exp(-D_lower[i, j])

# Print the sum of all p_ij
print("Sum of all p_ij: ", total_sum)

# Calculate p_12, p_13, p_23 (all are the same)
p_12 = exp(-A) / total_sum
print("p_12 = p_13 = p_23: ", p_12)

Sum of all p_ij:  3*exp(-A)
p_12 = p_13 = p_23:  1/3


### 18. Embedding three points into 1 dimension

#### Minimize KL

![Alt text](Images/minimize_kl.PNG)

In [291]:
# Constants
A = 1
B = np.sqrt(2)

# Distances
dp_12 = A
dp_13 = A
dp_23 = B

# Find each of the probabilities p_ij
total_sum = np.exp(-dp_12**2) + np.exp(-dp_13**2) + np.exp(-dp_23**2)
p_12 = np.exp(-dp_12**2) / total_sum
p_13 = np.exp(-dp_13**2) / total_sum
p_23 = np.exp(-dp_23**2) / total_sum
print("p_12: ", p_12)
print("p_13: ", p_13)
print("p_23: ", p_23)

# Symbolic variables
a = Symbol("a", real=True, positive=True)

# Distances in the lower dimensional space (1 dimension)
dq_12 = a
dq_13 = a
dq_23 = 2*a

# Find each of the probabilities q_ij
total_sum = sp.exp(-dq_12**2) + sp.exp(-dq_13**2) + sp.exp(-dq_23**2)
q_12 = sp.exp(-dq_12**2) / total_sum
q_13 = sp.exp(-dq_13**2) / total_sum
q_23 = sp.exp(-dq_23**2) / total_sum
print("q_12: ", q_12)
print("q_13: ", q_13)
print("q_23: ", q_23)
print()

# Calculate the KL expression
KL_pq = ( p_12 * sp.ln(p_12 / q_12) ) + ( p_13 * sp.ln(p_13 / q_13) )+ ( p_23 * sp.ln(p_23 / q_23) )
print("KL expression: ", KL_pq.simplify())

# Solve for a
solution = sp.solveset(KL_pq.simplify(), a)
print("Sympy can only find solutions using complex numbers. So no solution is shown")
print()

# How about this trick? 
# The kullback-leibler divergence is set equal to 0. This happens if each of the 
# natural logarithms in the KL expression are equal to 0. This happens if the value
# inside the logarithm is equal to 1. This happens whenever the numerator and denominator
# of the logarithm are equal, so, by setting the numerator and denominator equal to each other,
# we can solve for a. The values of (p_12 / q_12) and (p_23 / q_23) are equal to each other,
# so if we get the same value for a for the (p_12 / q_12) equation and the (p_23 / q_23) equation,
# then we have found the correct value for a.
sol_a1 = sp.solve(q_12 - p_12, a)
sol_a2 = sp.solve(q_23 - p_23, a)

# Are the two solutions equal?
if sol_a1 == sol_a2 and len(sol_a1) > 0:
    print( "a = ", sol_a1[0] )
    print("What value do we get for (p_12 / q_12) and (p_23 / q_23)?")
    print("p_12 / q_12: ", p_12 / q_12.subs(a, sol_a1[0]) )
    print("p_23 / q_23: ", p_23 / q_23.subs(a, sol_a1[0]) )
else:
    print("No solution found. Solution may be empty")

p_12:  0.42231879825151825
p_13:  0.42231879825151825
p_23:  0.15536240349696354
q_12:  exp(-a**2)/(2*exp(-a**2) + exp(-4*a**2))
q_13:  exp(-a**2)/(2*exp(-a**2) + exp(-4*a**2))
q_23:  exp(-4*a**2)/(2*exp(-a**2) + exp(-4*a**2))

KL expression:  -2.53391278950911*a**2 + log(0.723098356255259*(exp(3*a**2) + 0.5)**1.0)
Sympy can only find solutions using complex numbers. So no solution is shown

a =  0.577350269189625
What value do we get for (p_12 / q_12) and (p_23 / q_23)?
p_12 / q_12:  1.00000000000000
p_23 / q_23:  0.999999999999997


### 19. T-SNE

![Alt text](Images/t-sne.PNG)

Simplified definition because the real definition includes the variances for each of the dimensions. Here we just assume a unit variance for simplicity sake.

#### t-SNE First Example

In [292]:
# Constants
A = 1
B = np.sqrt(2)

# Distances
dp_12 = A
dp_13 = A
dp_23 = B

# Find each of the probabilities p_ij
total_sum = np.exp(-dp_12**2) + np.exp(-dp_13**2) + np.exp(-dp_23**2)
p_12 = np.exp(-dp_12**2) / total_sum
p_13 = np.exp(-dp_13**2) / total_sum
p_23 = np.exp(-dp_23**2) / total_sum
print("p_12: ", p_12)
print("p_13: ", p_13)
print("p_23: ", p_23)

# Symbolic variables
a = Symbol("a", real=True, positive=True)

# Distances in the lower dimensional space (1 dimension)
dq_12 = a
dq_13 = a
dq_23 = 2*a

# Find each of the probabilities q_ij
total_sum = (1 / (1 + dq_12**2)) + (1 / (1 + dq_13**2)) + (1 / (1 + dq_23**2))
q_12 = (1 / (1 + dq_12**2)) / total_sum
q_13 = (1 / (1 + dq_13**2)) / total_sum
q_23 = (1 / (1 + dq_23**2)) / total_sum
print("q_12: ", q_12.simplify())
print("q_13: ", q_13.simplify())
print("q_23: ", q_23.simplify())
print()

# Solve q_12 = p_12 or q_23 = p_23
sol_a1 = sp.solve(q_12 - p_12, a)
sol_a2 = sp.solve(q_23 - p_23, a)
print("a1 = ", sol_a1)
print("a2 = ", sol_a2)

p_12:  0.42231879825151825
p_13:  0.42231879825151825
p_23:  0.15536240349696354
q_12:  (4*a**2 + 1)/(3*(3*a**2 + 1))
q_13:  (4*a**2 + 1)/(3*(3*a**2 + 1))
q_23:  (a**2 + 1)/(3*(3*a**2 + 1))

a1 =  [1.15784634184208]
a2 =  [1.15784634184208]
